# 선형 회귀 학습 심화
## 하이퍼 파라미터 튜닝(GridSearchCV)
- 가장 높은 성능으로 학습하게 하는 하이퍼 파라미터 탐색 필요
- learning rate 및 정규화 정도에 따라 성능 차이 존재
## 데이터 전처리
## 파이프라인

## 검증 데이터셋 + for문

In [23]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [24]:
l_eta0 = [0.0001, 0.001, 0.01, 0.1]
l_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

In [25]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1234)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.25,
                                                  random_state=1234)
best_R2 = 0
for eta0 in l_eta0:
    for alpha in l_alpha:
        reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=eta0, alpha=alpha, random_state=1234)
        reg.fit(X_val, y_val)
        current_R2 = reg.score(X_val, y_val)
        if current_R2 > best_R2:
            best_R2 = current_R2
            best_eta0 = eta0
            best_alpha = alpha
reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=best_eta0, alpha=best_alpha, random_state=1234)
reg = reg.fit(X_train, y_train)
test_R2 = reg.score(X_test, y_test)

print('최적 러닝레이트:',best_eta0)
print('최적 정규화 계수:',best_alpha)
print('검증 데이터셋 R2:',best_R2)
print('테스트 데이터셋 R2:',test_R2)


최적 러닝레이트: 0.0001
최적 정규화 계수: 0.0001
검증 데이터셋 R2: 0.5521356829103785
테스트 데이터셋 R2: 0.4731219201091438


## GridSearchCV(Cross Validation)
- 교차 검증 활용: 검증 데이터셋을 여러 조합으로 평가한 후 score 평균
- KFold 함수: 교차 검증을 위해 사용, validator 객체 생성
- e.g. 라쏘 회귀에서 어떤 alpha 값이 가장 좋은지, 어떤 eta0 값이 가장 좋은지 -> 총 28가지의 경우 중 어떤 조합이 가장 좋은지

### GrideSerchCV 객체로 fit을 할 시 KFold 교차 검증 + for문 반복 실험으로 최적의 파라미터 조합을 찾아서 fit
- best_params_: 가장 좋은 조합의 하이퍼 파라미터 반환
- best_score_: 가장 좋은 조합의 스코어 반환

In [26]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1234)

# 중요!!
# alpha와 eta0 28가지의 조합
param_grid = {'alpha': l_alpha, 'eta0': l_eta0}
# 회귀 모델 객체가 estimator에 들어감
# params_grid는 딕셔너리 형태로 넣어줌
# 4-Fold(4겹 교차 검증)
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)
grid_search = GridSearchCV(estimator=SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', random_state=1234), param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)
y_pred_test = grid_search.predict(X_test)
test_MAE = np.abs(y_pred_test - y_test).mean()
print('최적 러닝레이트: ', grid_search.best_params_['eta0'])
print('최적 정규화 계수: ', grid_search.best_params_['alpha'])
print('최적 모델의 교차검증 스코어: ', grid_search.best_score_)
print('테스트 데이터셋 R2: ', grid_search.score(X_test,y_test))

최적 러닝레이트:  0.0001
최적 정규화 계수:  0.0001
최적 모델의 교차검증 스코어:  0.48593053352963667
테스트 데이터셋 R2:  0.4723402669032044


### 실습 1. tips data에서 tip을 나머지 피처로 릿지회귀 하는 문제에서, 최적의 하이퍼파라미터를 찾으세요. (4-겹 교차검증을 통해  l_alpha, l_eta0 중에서 최적의 alpha와 eta0를 찾으세요.)

In [27]:
import pandas as pd

tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [28]:
tips['smoker'] = tips['smoker'].map({'No': 0, 'Yes': 1})
tips['day'] = tips['day'].map({'Thur': 0, 'Fri': 1, 'Sat': 2, 'Sun': 3})
tips['time'] = tips['time'].map({'Lunch': 0, 'Dinner': 1})
X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns='tip'), tips.tip, test_size=0.2, random_state=1234)

In [29]:
param_grid = {'alpha': l_alpha, 'eta0': l_eta0}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)
grid_search = GridSearchCV(estimator=SGDRegressor(penalty='l2', max_iter=100000, random_state=1234, learning_rate='constant'), param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)
print('최적 eta0(learning rate): ', grid_search.best_params_['eta0'])
print('최적 alpha(정규화 계수): ', grid_search.best_params_['alpha'])

최적 eta0(learning rate):  0.0001
최적 alpha(정규화 계수):  0.0001


## 데이터 전처리
- 결측치 처리
- 스케일링
- 범주형 데이터 처리
#### 여기까지는 scikit-learn에서 처리 객체 제공

- 데이터 병합
- feature 선택
#### 처리 객체를 파이프라인처럼 쌓아서 전처리 및 선형 모델을 하나의 파이프라인으로 함수화 가능

### 결측치 처리 객체: imputer=SimpleImputer()
- fit: X를 통해 변환기가 fit 수행
- transform: 결측치를 fit 한 결과를 통해 채움
- fit_transform: fit과 transform 동시 수행

In [30]:
from sklearn.impute import SimpleImputer

X, y = load_diabetes(return_X_y=True, as_frame=True)
X.iloc[np.random.randint(0, X.shape[0], 10), np.random.randint(0, X.shape[1], 3)] = np.nan
X.isna().sum()

age     0
sex     0
bmi     0
bp      0
s1      0
s2      0
s3      0
s4      0
s5     10
s6     10
dtype: int64

In [31]:
X_mean = X.mean()

In [32]:
X = X.fillna(X.mean())
X.isna().sum()

age    0
sex    0
bmi    0
bp     0
s1     0
s2     0
s3     0
s4     0
s5     0
s6     0
dtype: int64

In [33]:
imputer = SimpleImputer()
imputer.fit(X)
X = imputer.transform(X)
np.sum(X == np.nan)

np.int64(0)

In [34]:
Y = np.full_like(X, np.nan)
Y = imputer.transform(Y)
Y - np.array(X_mean)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  4.33680869e-19, -2.43945489e-19]])